In [44]:
# import the necesary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [45]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ss = pd.read_csv('sample_submission.csv')

In [46]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [47]:
train = train.drop(['PassengerId', 'Cabin', 'Name'], axis=1)
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [5]:
for i in train.columns:
    if train[i].dtypes=='object':
        train[i] = train[i].fillna(train[i].mode()[0])
    else:
        train[i] = train[i].fillna(train[i].mean())

In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
for i in train.columns:
    if train[i].dtypes=='object':
        train[i] = ohe.fit_transform(train[[i]]).toarray()

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['CryoSleep'] = le.fit_transform(train['CryoSleep'])
train['VIP'] = le.fit_transform(train['VIP'])

In [8]:
x = train.drop('Transported',axis=1)
y = train['Transported']

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [51]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 150,random_state=42,max_depth=12,oob_score=True)
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=12, n_estimators=150, oob_score=True,
                       random_state=42)

In [52]:
y_pred = rf.predict(x_test)
y_pred

array([False,  True, False, ...,  True, False, False])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7837837837837838

In [54]:
from sklearn import neighbors
from math import sqrt

knn = neighbors.KNeighborsClassifier(n_neighbors=5,p=2)
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [55]:
y_pred_knn = knn.predict(x_test)
y_pred_knn

array([False, False, False, ...,  True, False, False])

In [56]:
knn_acc = accuracy_score(y_test,y_pred_knn)
knn_acc

0.7642323174238068

In [57]:
for i in test.columns:
    if test[i].dtypes=='object':
        test[i] = test[i].fillna(test[i].mode()[0])
    else:
        test[i] = test[i].fillna(test[i].mean())

In [58]:
b = test['PassengerId']

In [59]:
test.drop(['PassengerId','Cabin', 'Name',],axis=1,inplace=True)

In [32]:
from sklearn import preprocessing
for i in test.columns:
    if test[i].dtype=='object':
        label_encoder=preprocessing.LabelEncoder()
        test[i]=label_encoder.fit_transform(test[i])

In [33]:
y_valid = rf.predict(test)
y_valid

array([ True, False,  True, ...,  True,  True,  True])

In [34]:
michael_olaoyo = pd.DataFrame({"PassengerId": b, "Transported":y_valid})
michael_olaoyo

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [35]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

boost_model = XGBClassifier(n_jobs=-1, random_state=42,max_depth = 5)

#Fitting the model
boost_model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [36]:
pred = boost_model.predict(x_test)
pred

array([0, 1, 0, ..., 1, 0, 0])

In [37]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm = confusion_matrix(y_test,pred)
xg_acc=accuracy_score(y_test,pred)
print(classification_report(y_test,pred))
print(cm)
print(xg_acc)

              precision    recall  f1-score   support

       False       0.79      0.75      0.77       863
        True       0.77      0.81      0.79       876

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739

[[647 216]
 [170 706]]
0.7780333525014376


In [39]:
y_valid_xg = boost_model.predict(test)
y_valid_xg

array([1, 0, 1, ..., 1, 1, 1])

In [40]:
spaceship_xg = pd.DataFrame({"PassengerId": b, "Transported":y_valid_xg})
spaceship_xg

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1
...,...,...
4272,9266_02,1
4273,9269_01,1
4274,9271_01,1
4275,9273_01,1


In [43]:
spaceship_xg['Transported'] =spaceship_xg['Transported'].apply(lambda x:True if x==1 else False)
spaceship_xg

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [62]:
michael_olaoyo.to_csv('desktop', index=False)